In [1]:
import gc
import json
import matplotlib.pyplot as plt
import numpy as np
import os
from sklearn.manifold import TSNE
import sys
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

%load_ext autoreload
%autoreload 2
sys.path.append('../act/')
import steering_vectors
import utils

### Generating data
This is the setup I used to generate the data. You can skip this if you want to directly load the final dataset. 

##### Data description 

We create datasets by prompting an advanced model (GPT4) to output notes written in specific styles. We thus consider this advanced model to be the oracle of what makes the ideal note structure. 

We keep the prompt we used on GPT4 and use it as a baseline to obtain a prompt-based result on Phi3.5. We then perform i) few-shot learning and ii) representation optimisation. We compare the performances of all against the prompt-based approach. 

We generate two types of notes: 
- [1] One popular prompt structure leverages the COSTAR framework introduced in https://towardsdatascience.com/how-i-won-singapores-gpt-4-prompt-engineering-competition-34c195a93d41. We use this setting to generate a pitch summary. 
- [2] Personality-based notes. We generate notes in the personalities of David Sacks, Tina Fey, and Winston Churchill. 

We use the TED talk transcripts as our main dataset for which we want to obtain notes. Followed by i) an instruction to summarise the prompt with prompts as in [1] and [2], ii) few-shot examples of prompts and iii) representation finetuning. 

For COSTAR we use: 
```
Role: You are an analyst reviewing TED Talks to extract key insights and ideas.
Objective: Summarize the TED Talk transcript into a concise and engaging note.
Context: Imagine this summary will be shared with a professional audience who may not have the time to watch the talk themselves. They rely on your summary to decide if they should invest their time in it. Ensure your note captures the essence of the talk.
Task:
- Identify the main theme of the talk.
- Highlight the speaker's key arguments or ideas.
- Summarize any standout examples, anecdotes, or evidence provided.
- Include the speaker’s core message or "call to action."
- Write your note in clear, accessible language, ensuring it can be understood by someone with no prior knowledge of the subject.
Audience: Professionals, executives, and curious learners looking for actionable insights or new perspectives.
Result: A polished, compelling summary note (100–150 words) that encapsulates the essence of the TED Talk.
```

For the personality-based notes we use: ```Generate a summary of this transcript in the style of {}``` followed by a *descriptive* prompts to guide the model even more: 
- David Sacks: ```A concise, no-nonsense summary that highlights the core idea, actionable insights, and strategic implications, presented in a structured and analytical format.```
- Tina Fey: ```A witty, irreverent summary filled with clever asides and humor, while still capturing the talk’s main points with a relatable and engaging tone.```
- Winston Churchill: ```A grand, eloquent summary that frames the talk as a momentous contribution to human progress, delivered with formal rhetoric and a touch of inspiring gravitas.```

In [ ]:
from openai import OpenAI
import pandas as pd
client = OpenAI(api_key="...")

# load transcripts
df = pd.read_csv('../data/styles/ted_talks_en.csv')
transcripts = df['transcript']

# filter out transcripts that are longer than 1000 and less than 8000 tokens (1 tok \approx 4 chars)
transcripts = [tr for tr in transcripts if len(tr) < 32000 and len(tr) > 4000]

# This cell is optional and should be ran only if you want to generate the data yourself
# let's generate some data with openai 
USER_PROMPT_COSTAR = '''
Role: You are an analyst reviewing TED Talks to extract key insights and ideas. \n
Objective: Summarize the TED Talk transcript into a concise and engaging note. \n
Context: Imagine this summary will be shared with a professional audience who may not have the time to watch the talk themselves. They rely on your summary to decide if they should invest their time in it. Ensure your note captures the essence of the talk. \n 
Task: \n
- Identify the main theme of the talk.\n
- Highlight the speaker's key arguments or ideas.\n
- Summarize any standout examples, anecdotes, or evidence provided.\n
- Include the speaker’s core message or "call to action."\n
- Write your note in clear, accessible language, ensuring it can be understood by someone with no prior knowledge of the subject.\n
Audience: Professionals, executives, and curious learners looking for actionable insights or new perspectives.\n
Result: A polished, compelling summary note (100–150 words) that encapsulates the essence of the TED Talk.\n
\n 
TED Talk Transcript: \n
'''
USER_PROMPT_PERSONALITY_DS = '''
Generate a summary (100-150 words) of the given transcript in the style of David Sacks. It should be a concise, no-nonsense summary that highlights the core idea, actionable insights, and strategic implications, presented in a structured and analytical format.\n
Transcript: \n
'''
USER_PROMPT_PERSONALITY_TF = '''
Generate a summary (100-150 words) of the given transcript in the style of Tina Fey. It should a witty, irreverent summary filled with clever asides and humor, while still capturing the talk’s main points with a relatable and engaging tone.\n
Transcript: \n
'''
USER_PROMPT_PERSONALITY_WC = '''
Generate a summary (100-150 words) of the given transcript in the style of Winston Churchill. It should be a grand, eloquent summary that frames the talk as a momentous contribution to human progress, delivered with formal rhetoric and a touch of inspiring gravitas.\n
Transcript: \n
'''

prompts_costar = []
prompts_personality = {'ds': [], 'tf': [], 'wc': []}
outputs_costar = []
outputs_personality = {'ds': [], 'tf': [], 'wc': []}

for it in range(50): 
        resp_1 = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": USER_PROMPT_COSTAR + " " + transcripts[it]}],
                temperature = 0.6
                )
        resp_2 = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": USER_PROMPT_PERSONALITY_DS + " " + transcripts[it]}],
                temperature = 0.6
                )
        resp_3 = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": USER_PROMPT_PERSONALITY_TF + " " + transcripts[it]}],
                temperature = 0.6
                )
        resp_4 = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=[{"role": "user", "content": USER_PROMPT_PERSONALITY_WC + " " + transcripts[it]}],
                temperature = 0.6
                )
        outputs_costar.append(resp_1.choices[0].message.content)
        outputs_personality['ds'].append(resp_2.choices[0].message.content)
        outputs_personality['tf'].append(resp_3.choices[0].message.content)
        outputs_personality['wc'].append(resp_4.choices[0].message.content)
        prompts_costar.append(USER_PROMPT_COSTAR + " " + transcripts[it])
        prompts_personality['ds'].append(USER_PROMPT_PERSONALITY_DS + " " + transcripts[it])
        prompts_personality['tf'].append(USER_PROMPT_PERSONALITY_TF + " " + transcripts[it])
        prompts_personality['wc'].append(USER_PROMPT_PERSONALITY_WC + " " + transcripts[it])

# save the data
import json

data = {
        'prompts_costar': prompts_costar,
        'prompts_personality': prompts_personality,
        'outputs_costar': outputs_costar,
        'outputs_personality': outputs_personality
        }

with open('../data/styles/note_styles.json', 'w') as f:
        json.dump(data, f)

### Let's begin!

In [42]:
# Load the model we will use 
device = 'cuda'
model_name = 'microsoft/Phi-3.5-mini-instruct'
# model_name = 'meta-llama/Llama-3.2-3B'

# Load Phi model 
model = AutoModelForCausalLM.from_pretrained(
    model_name, #
    trust_remote_code=True, 
    torch_dtype=torch.bfloat16
)
# model = AutoModelForCausalLM.from_pretrained(model_name)
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def tokenize_instructions(tokenizer, instructions):
    return tokenizer.apply_chat_template(
        instructions,
        padding=True,
        truncation=False,
        return_tensors="pt",
        return_dict=True,
        add_generation_prompt=True,
    ).input_ids

# Generating notes in a given style 

AI-based note-taking apps record your meeting, transcribe it and consequently create compact notes that summarize all relevant information. Or well, you *hope* that the summary reliably extracts the necessary information from the transcript. To achieve this it often requires quite some iterations in order to converge on a proper prompt structure. 

Some example note styles: 
- https://help.fathom.video/en/articles/640768 allows users to have access to 14 templates ranging from sales notes to one-on-one meeting summaries. 
- https://www.granola.ai provides 5 templates including one-on-one, pitch, user interview and standup. 
- https://www.plaud.ai offers notes such as SOAP for therapy purposes and lecture summaries. 

We'd like to avoid the use of prompts and explore alternatives that can implicitly extract a user's style preference based on other notes they liked. 
Approaches to achieve this can be few-shot prompting, finetuning, parameter-efficient finetuning. Here, I consider a different approach: a finetuning method on the level of representations. 

In [5]:
# load csv dataset 
import pandas as pd

df = pd.read_csv('../data/styles/ted_talks_en.csv')
transcripts = df['transcript']

# filter out transcripts that are longer than 1000 and less than 8000 tokens (1 tok \approx 4 chars)
transcripts = [tr for tr in transcripts if len(tr) < 32000 and len(tr) > 4000]

with open('../data/styles/note_styles.json', 'r') as f:
        data = json.load(f)

prompts_costar = data['prompts_costar'] # used to generate data
prompts_personality = data['prompts_personality'] # used to generate data
outputs_costar = data['outputs_costar']
outputs_personality = data['outputs_personality']

# Let's use the first n_train examples to train the model
n_train = 45# Load the data 


In [6]:
# Let's check whether our summaries make any sense 
# print(transcripts[0])
print(f"Summary Costar: {outputs_costar[49]}")
print(f"Summary David Sacks: {outputs_personality['ds'][49]}")
print(f"Summary Tina Fey: {outputs_personality['tf'][49]}")
print(f"Summary Winston Churchill: {outputs_personality['wc'][49]}")

Summary Costar: In this thought-provoking TED Talk, renowned astronomer and cosmologist explores the profound interconnectedness of our universe, from the microcosm of quantum physics to the vast cosmos. The speaker emphasizes that while we have made strides in understanding the universe's origins—tracing back to the Big Bang—we still grapple with the fundamental question of why it occurred. He highlights the dual challenge of synthesizing our knowledge of the very small (quantum mechanics) and the very large (cosmology), as well as comprehending the complexity of human existence itself.

Drawing on examples like the diverse ecosystems of planets and the impact of human activity on Earth, he warns of the unprecedented challenges we face in the 21st century. Emerging technologies hold immense potential but also pose risks that could have catastrophic consequences. The speaker calls for a new generation of scientists to take moral responsibility for the implications of their work, echoin

### Prompting baseline
We begin with establishing a baseline of performance using the same prompts we used to generate the data. We assume the generated data (from GPT4o) is our golden baseline of note styles. Here we test in what way a different model is able to replicate the style using solely the describing prompt. 

One can imagine this to test the following: a human wrote notes in its style; we take our best stab at describing the note and optimising a prompt, and we test it on some model. 

In [8]:
# Iterate over the test transcripts and generate summaries in the different styles 
print('\n\n\nPrompt performance for COSTAR note\n\n\n')
for prompt in prompts_costar[n_train:]: 
    toks = tokenize_instructions(tokenizer, [{"role": "user", "content": prompt}]).to(device)
    output = model.generate(toks, max_new_tokens = 400)
    summary = tokenizer.decode(output[0,toks.shape[1]:], skip_special_tokens=True)
    print(summary)
    del toks, output, summary
    torch.cuda.empty_cache()
    gc.collect()

You are not running the flash-attention implementation, expect numerical differences.





Prompt performance for COSTAR note



**Summary Note:**

In a TED Talk, the speaker explores the burgeoning urban landscapes of squatter communities, which are rapidly evolving into the cities of tomorrow. With an estimated 1.5 billion squatters globally, these informal settlements are home to a significant portion of the world's population, with projections indicating a rise to three billion by 2050.

The speaker shares vivid accounts from various squatter communities in Nairobi, Mumbai, Rio de Janeiro, and Istanbul, highlighting the resilience and ingenuity of residents who create vibrant, self-sufficient neighborhoods despite lacking basic services. These communities, often overlooked by formal urban planning, are characterized by their strong sense of community, resourcefulness, and the ability to provide for their own needs through informal economies and self-built infrastructure.

Central to the speaker's message is the concept of "security of tenure" and the importance of rec

In [14]:
base_prompt = '''
Generate a summary (100-150 words) of the given transcript. It should be a concise, no-nonsense summary that highlights the core idea, actionable insights, and strategic implications, presented in a structured and analytical format.\n
Transcript: \n
'''
print('\n\n\nPrompt performance for David Sacks style note\n\n\n')
for ts in range(0,5):
    toks = tokenize_instructions(tokenizer, [{"role": "user", "content": base_prompt + transcripts[n_train+ts]}]).to(device)
    output = model.generate(toks, max_new_tokens = 400)
    summary = tokenizer.decode(output[0,toks.shape[1]:], skip_special_tokens=True)
    print('PROMPT-BASED SUMMARY '+str(ts))
    print(summary)
    del toks, output, summary
    torch.cuda.empty_cache()
    gc.collect()




Prompt performance for David Sacks style note



PROMPT-BASED SUMMARY 0
Summary:

The transcript explores the emergence of informal urban settlements, often referred to as "squatter communities," as the cities of tomorrow. The speaker highlights the rapid urban migration, with millions of people moving from rural to urban areas each week, leading to the proliferation of these communities worldwide.

The speaker shares vivid examples from Nairobi, Mumbai, Rio de Janeiro, and Istanbul, illustrating the stark contrast between the harsh living conditions and the resilient spirit of the residents. Despite lacking basic amenities like water, sanitation, and electricity, these communities exhibit a strong sense of freedom and self-reliance.

The speaker emphasizes the need for engagement and recognition of these communities as legitimate forms of urban development. The speaker advocates for security of tenure and access to politics, citing successful examples from Turkey where squatters ha

In [15]:
base_prompt = '''
Generate a summary (100-150 words) of the given transcript. It should a witty, irreverent summary filled with clever asides and humor, while still capturing the talk’s main points with a relatable and engaging tone.\n
Transcript: \n
'''
print('\n\n\nPrompt performance for Tina Fey style note\n\n\n')
for ts in range(0,5): 
    toks = tokenize_instructions(tokenizer, [{"role": "user", "content": base_prompt + transcripts[n_train+ts]}]).to(device)
    output = model.generate(toks, max_new_tokens = 400)
    summary = tokenizer.decode(output[0,toks.shape[1]:], skip_special_tokens=True)
    print('PROMPT-BASED SUMMARY '+str(ts))
    print(summary)
    del toks, output, summary
    torch.cuda.empty_cache()
    gc.collect()




Prompt performance for Tina Fey style note



PROMPT-BASED SUMMARY 0
Summary:

In a witty exploration of the world's burgeoning urban squatter communities, we're invited to rethink our notions of progress and prosperity. Julius, a newcomer to Nairobi's Kibera, encapsulates the harsh reality of life in these sprawling informal settlements, where basic amenities are luxuries and the struggle for survival is a daily reality.

From the squalid conditions of Southland in Nairobi to the makeshift homes of Sultanbelyi in Istanbul, these communities are not just surviving; they're thriving, with their own unique systems of governance, commerce, and social organization. Despite the challenges, residents like Geeta Jiwa and Nine, the "grandma" of Sultanbelyi, are carving out a sense of community and belonging, proving that these neighborhoods are not just places of poverty, but the very fabric of urban development.

The talk underscores the need for engagement and recognition of these communi

In [16]:
base_prompt = '''
Generate a summary (100-150 words) of the given transcript. It should be a grand, eloquent summary that frames the talk as a momentous contribution to human progress, delivered with formal rhetoric and a touch of inspiring gravitas.\n
Transcript: \n
'''
print('\n\n\nPrompt performance for Winston Churchill style note\n\n\n')
for ts in range(0,5): 
    toks = tokenize_instructions(tokenizer, [{"role": "user", "content": base_prompt + transcripts[n_train+ts]}]).to(device)
    output = model.generate(toks, max_new_tokens = 400)
    summary = tokenizer.decode(output[0,toks.shape[1]:], skip_special_tokens=True)
    print('PROMPT-BASED SUMMARY '+str(ts))
    print(summary)
    del toks, output, summary
    torch.cuda.empty_cache()
    gc.collect()




Prompt performance for Winston Churchill style note



PROMPT-BASED SUMMARY 0
In this grand exposition, we traverse the landscapes of burgeoning urbanity, where the squatter communities of Nairobi, Bombay, Rio de Janeiro, and Istanbul stand as the vanguard of tomorrow's metropolises. These enclaves, often dismissed as mere slums, are in fact the crucibles of innovation and resilience, housing over a billion souls who, against the odds, forge a life amidst the squalor.

Julius, a denizen of Kibera, encapsulates the spirit of these communities. His narrative, a testament to the daily odyssey of millions migrating from rural obscurity to urban promise, underscores the urgency of our discourse. The statistics are stark: a billion squatters, a burgeoning populace that will swell to three billion by 2050. Yet, within these numbers lies a paradoxical truth—the very essence of urban potential.

The squatter settlements, with their rudimentary shelters and makeshift amenities, are not mere r

### Few-shot prompting 
Here we pass several note examples in the context to have the model adjust the style of the given transcript. 

In [17]:
num_shots = 5
few_shots_costar = ' '.join(['Example '+str(p)+': ' + outputs_costar[:num_shots][p] for p in range(num_shots)])
few_shots_ds = ' '.join(['Example '+str(p)+': ' + outputs_personality['ds'][:num_shots][p] for p in range(num_shots)])
few_shots_tf = ' '.join(['Example '+str(p)+': ' + outputs_personality['tf'][:num_shots][p] for p in range(num_shots)])
few_shots_wc = ' '.join(['Example '+str(p)+': ' + outputs_personality['wc'][:num_shots][p] for p in range(num_shots)])
base_prompt = '''You are given several examples of summary notes of transcripts. Generate a summary note of 100-150 words that fits with the style. \n Examples: \n ''' 

In [18]:
# Let's get the outputs for the test set 
print('COSTAR STYLE')
for tr in range(0,5): 
    print('BASELINE SUMMARY')
    print(outputs_costar[n_train+tr])
    prompt = base_prompt + few_shots_costar + '\n Transcript: ' + transcripts[n_train+tr]
    toks = tokenize_instructions(tokenizer, [{"role": "user", "content": prompt}]).to(device)
    output = model.generate(toks, max_new_tokens = 400)
    print('FEW-SHOT SUMMARY')
    summary = tokenizer.decode(output[0,toks.shape[1]:], skip_special_tokens=True)
    print(summary)
    del toks, output, summary
    torch.cuda.empty_cache()
    gc.collect()

COSTAR STYLE
BASELINE SUMMARY
In this compelling TED Talk, the speaker explores the emergence of informal urban communities, or "squatter cities," as the future of urban living. Highlighting the rapid migration of millions to urban areas, he presents a stark reality: by 2050, an estimated three billion people will reside in these informal settlements. Through the poignant story of Julius, a newcomer to Kibera, and the experiences of residents like Armstrong, the speaker illustrates how these communities, often overlooked, possess resilience and a unique quality of life that challenges conventional notions of prosperity.

The core message emphasizes the necessity for cities to recognize and engage with these neighborhoods, advocating for security of tenure and political access. By fostering an inclusive approach, urban planners can harness the potential of these vibrant communities, ultimately shaping the cities of tomorrow. The speaker's call to action is clear: we must shift our perce

In [22]:
print('DAVID SACKS STYLE')
for tr in range(0,5):
    print('BASELINE SUMMARY')
    print(outputs_personality['ds'][n_train+tr])
    prompt = base_prompt + few_shots_ds + '\n Transcript: ' + transcripts[n_train+tr]
    toks = tokenize_instructions(tokenizer, [{"role": "user", "content": prompt}]).to(device)
    output = model.generate(toks, max_new_tokens = 400)
    print('FEW-SHOT SUMMARY')
    summary = tokenizer.decode(output[0,toks.shape[1]:], skip_special_tokens=True)
    print(summary)
    del toks, output, summary
    torch.cuda.empty_cache()
    gc.collect()

DAVID SACKS STYLE
BASELINE SUMMARY
In the exploration of urbanization, the speaker highlights the emergence of informal communities, or "squatter neighborhoods," as the cities of tomorrow. With an alarming statistic of nearly 200,000 people migrating daily from rural areas to urban environments, these communities are projected to grow significantly by 2050, reaching three billion inhabitants. The speaker emphasizes the resilience and adaptability of these neighborhoods, showcasing their evolution from rudimentary structures to more developed living spaces. 

Key actionable insights include the necessity for security of tenure and community political engagement, rather than traditional property rights, to empower residents. The Turkish model, which protects squatters and allows them to establish community governance, serves as a potential framework for other regions. Ultimately, the strategic implication is clear: urban planning must recognize and integrate these informal settlements to

In [23]:
print('TINA FEY STYLE')
for tr in range(0,5):
    print('BASELINE SUMMARY')
    print(outputs_personality['tf'][n_train+tr])
    prompt = base_prompt + few_shots_tf + '\n Transcript: ' + transcripts[n_train+tr]
    toks = tokenize_instructions(tokenizer, [{"role": "user", "content": prompt}]).to(device)
    output = model.generate(toks, max_new_tokens = 400)
    print('FEW-SHOT SUMMARY')
    summary = tokenizer.decode(output[0,toks.shape[1]:], skip_special_tokens=True)
    print(summary)
    del toks, output, summary
    torch.cuda.empty_cache()
    gc.collect()

TINA FEY STYLE
BASELINE SUMMARY
Alright, folks, grab your hard hats because we’re diving into the concrete jungle of tomorrow, and it’s less about skyscrapers and more about squatter shacks! Our speaker takes us on a whirlwind tour through the world’s most vibrant “neighborhoods” — think Kibera, Rio’s favelas, and Istanbul’s Sultanbelyi — where the real estate market is as stable as a Jenga tower in an earthquake. With a staggering 1.5 million people migrating to urban areas every week, it’s clear that the future is crowded. But don’t let the corrugated metal roofs fool you; these communities are brimming with hope, resilience, and, apparently, an unshakeable bond with their trash heaps. Our hero, Julius, shows us that while they may lack flush toilets, they have something more valuable: a sense of community and a whole lot of spirit. So, let’s ditch the “urban poverty” label and recognize these bustling hubs as the legit cities of the future—because who needs a mortgage when you can j

In [24]:
print('WINSTON CHURCHILL STYLE')
for tr in range(0,5):
    print('BASELINE SUMMARY')
    print(outputs_personality['wc'][n_train+tr])
    prompt = base_prompt + few_shots_wc + '\n Transcript: ' + transcripts[n_train+tr]
    toks = tokenize_instructions(tokenizer, [{"role": "user", "content": prompt}]).to(device)
    output = model.generate(toks, max_new_tokens = 400)
    print('FEW-SHOT SUMMARY')
    summary = tokenizer.decode(output[0,toks.shape[1]:], skip_special_tokens=True)
    print(summary)
    del toks, output, summary
    torch.cuda.empty_cache()
    gc.collect()

WINSTON CHURCHILL STYLE
BASELINE SUMMARY
Ladies and gentlemen, gather 'round as we contemplate a momentous discourse on the cities of tomorrow, delivered with fervor and insight. In the heart of urban landscapes, from the sprawling slums of Kibera to the vibrant favelas of Rio, we are presented with an indomitable spirit of humanity. The speaker unveils the stark realities faced by millions, yet amidst the squalor, there blooms a resilience that defies the very constraints of material existence. As we witness the migration of nearly 1.5 million souls to these burgeoning metropolises each week, we are called to recognize that these communities represent not mere poverty, but a profound testament to human aspiration and ingenuity. The future lies not in the disdain for these neighborhoods but in our engagement with their inhabitants, as they forge their destinies and shape the cities of the future. Let us, therefore, embrace this opportunity to uplift and empower, for therein lies the tr

### Representation finetuning

##### COSTAR

In [9]:
# COSTAR
# to optimise the steering vector, we prepend the data with the instruction; instead of the transcript, as this will lead to a large token overhead, we just add in a endoftext token.
dd_with_c_A = [[{"role": "user", "content": "Generate a 100-150 word summary of the given transcript. Transcript: ..."}, {"role": "assistant", "content":y}] for y in outputs_costar[:n_train]]
toks_A = tokenize_instructions(tokenizer, dd_with_c_A).to(device)
# during optimisation, we plot the predicted tokens just to see how things are progressing
sv_optimised = steering_vectors.optimise_steering_vecs(model, toks_A, None, hidden_dim=3072, layer=15, batch_size=1, num_epochs=5)

0 11.609110832214355
1 10.039969444274902
2 10.066263198852539
3 5.626282691955566
4 11.830156326293945
5 10.46678638458252
6 2.657907485961914
7 10.711786270141602
8 10.4572172164917
9 8.274828910827637
10 5.9231133460998535
11 6.177060127258301
12 6.0334906578063965
13 8.981745719909668
14 8.084341049194336
15 6.3641862869262695
16 6.998361110687256
17 6.438157558441162
18 7.265748977661133
19 8.066332817077637
20 7.782523155212402
21 6.136069297790527
22 4.193398475646973
23 3.8030123710632324
24 5.502898693084717
25 5.7018890380859375
26 3.3318371772766113
27 3.4807777404785156
28 4.344520092010498
29 4.403027534484863
30 2.8099136352539062
31 4.5629496574401855
32 5.329823017120361
33 4.313325881958008
34 3.7871198654174805
35 4.100249290466309
36 4.798498153686523
37 4.215564250946045
38 2.835848808288574
39 4.193678379058838
40 3.723299741744995
41 4.686225414276123
42 4.403450965881348
43 4.508279800415039
44 3.8929834365844727
0 4.436291217803955
1 4.131062984466553
2 4.170880

In [10]:
# test the steering vector
base_prompt = "Generate a 100-150 word summary of the given transcript."
for tr in range(5):
    print('BASELINE SUMMARY\n')
    print(outputs_costar[n_train+tr])
    prompt = base_prompt + '\n Transcript: ' + transcripts[n_train+tr]
    toks_test = tokenize_instructions(tokenizer, [{"role": "user", "content": prompt}])
    # steer to A
    generation_intervene, _ = steering_vectors.do_steering(model, toks_test.to(device), sv_optimised.to(device), scale = -.5, layer = [15], proj=False, all_toks=False)
    intervene = tokenizer.decode(generation_intervene[0,toks_test.shape[1]:], skip_special_tokens=True)
    print('GENERATED SUMMARY\n')
    print(intervene)
    print('\n')

BASELINE SUMMARY

In this compelling TED Talk, the speaker explores the emergence of informal urban communities, or "squatter cities," as the future of urban living. Highlighting the rapid migration of millions to urban areas, he presents a stark reality: by 2050, an estimated three billion people will reside in these informal settlements. Through the poignant story of Julius, a newcomer to Kibera, and the experiences of residents like Armstrong, the speaker illustrates how these communities, often overlooked, possess resilience and a unique quality of life that challenges conventional notions of prosperity.

The core message emphasizes the necessity for cities to recognize and engage with these neighborhoods, advocating for security of tenure and political access. By fostering an inclusive approach, urban planners can harness the potential of these vibrant communities, ultimately shaping the cities of tomorrow. The speaker's call to action is clear: we must shift our perception and ac

##### David Sacks

In [13]:
# PERSONALITY DS
dd_with_c_A = [[{"role": "user", "content": "Generate a 100-150 word summary of the given transcript. Transcript: ..."}, {"role": "assistant", "content":y}] for y in outputs_personality['ds'][:n_train]]
toks_A = tokenize_instructions(tokenizer, dd_with_c_A).to(device)
sv_optimised = steering_vectors.optimise_steering_vecs(model, toks_A, None, hidden_dim=3072, layer=20, batch_size=1, num_epochs=5)

0 10.894625663757324
1 10.254132270812988
2 9.082740783691406
3 5.2620415687561035
4 8.139419555664062
5 9.454794883728027
6 6.617122650146484
7 6.115865707397461
8 7.394601821899414
9 7.407975673675537
10 3.561002254486084
11 5.8562750816345215
12 3.9967899322509766
13 4.756296634674072
14 4.24652624130249
15 3.866274833679199
16 4.056273937225342
17 3.8198561668395996
18 4.390946865081787
19 5.234565258026123
20 4.559864044189453
21 2.9207072257995605
22 3.5010273456573486
23 3.318793296813965
24 4.0341057777404785
25 3.1520488262176514
26 3.8681936264038086
27 3.463855028152466
28 3.3578145503997803
29 3.204442024230957
30 2.6577494144439697
31 2.8525006771087646
32 3.5141425132751465
33 3.66479754447937
34 2.7725579738616943
35 3.636850595474243
36 2.4839611053466797
37 3.2252933979034424
38 1.8190909624099731
39 2.680476427078247
40 2.3595781326293945
41 2.7275424003601074
42 2.6707534790039062
43 2.612300157546997
44 2.8722050189971924
0 3.0261478424072266
1 3.169865369796753
2 3

In [14]:
# test the steering vector
base_prompt = "Generate a 100-150 word summary of the given transcript."
for tr in range(0,5): 
    print('BASELINE SUMMARY\n')
    print(outputs_personality['ds'][n_train+tr])
    prompt = base_prompt + '\n Transcript: ' + transcripts[n_train+tr]
    toks_test = tokenize_instructions(tokenizer, [{"role": "user", "content": prompt}])
    # steer to A
    generation_intervene, _ = steering_vectors.do_steering(model, toks_test.to(device), sv_optimised.to(device), scale = -1, layer = [20], proj=False, all_toks=False)
    intervene = tokenizer.decode(generation_intervene[0,toks_test.shape[1]:], skip_special_tokens=True)
    print('GENERATED SUMMARY\n')
    print(intervene)
    print('\n')

BASELINE SUMMARY

In the exploration of urbanization, the speaker highlights the emergence of informal communities, or "squatter neighborhoods," as the cities of tomorrow. With an alarming statistic of nearly 200,000 people migrating daily from rural areas to urban environments, these communities are projected to grow significantly by 2050, reaching three billion inhabitants. The speaker emphasizes the resilience and adaptability of these neighborhoods, showcasing their evolution from rudimentary structures to more developed living spaces. 

Key actionable insights include the necessity for security of tenure and community political engagement, rather than traditional property rights, to empower residents. The Turkish model, which protects squatters and allows them to establish community governance, serves as a potential framework for other regions. Ultimately, the strategic implication is clear: urban planning must recognize and integrate these informal settlements to harness their po

##### Tina Fey

In [40]:
# PERSONALITY TF
dd_with_c_A = [[{"role": "user", "content": "Generate a 100-150 word summary of the given transcript. \n Transcript: " + x[:200]+'...'+x[-200:]}, {"role": "assistant", "content":y}] for (x,y) in zip(transcripts[:n_train],outputs_personality['tf'][:n_train])]
toks_A = tokenize_instructions(tokenizer, dd_with_c_A).to(device)
sv_optimised = steering_vectors.optimise_steering_vecs(model, toks_A, None, hidden_dim=3072, layer=20, batch_size=6, num_epochs=10)

0 7.661734580993652
6 8.179352760314941
12 5.684007167816162
18 6.355134963989258
24 7.709468364715576
30 7.051886081695557
36 5.009082317352295
42 7.331071853637695
0 5.961174011230469
6 6.321486949920654
12 4.545039176940918
18 4.971362113952637
24 5.845616817474365
30 5.504319667816162
36 4.05129861831665
42 5.690041542053223
0 4.800117015838623
6 5.097999572753906
12 3.8710296154022217
18 4.308742046356201
24 5.009949207305908
30 4.830680847167969
36 3.6466896533966064
42 4.903584003448486
0 4.30657958984375
6 4.568336009979248
12 3.5658440589904785
18 3.9455580711364746
24 4.485010147094727
30 4.394354820251465
36 3.379486322402954
42 4.468293190002441
0 3.979707956314087
6 4.22817325592041
12 3.373595714569092
18 3.729187488555908
24 4.194112777709961
30 4.135262489318848
36 3.2070865631103516
42 4.142096042633057
0 3.7557523250579834
6 3.9799232482910156
12 3.2247567176818848
18 3.561129570007324
24 3.9650745391845703
30 3.927382707595825
36 3.0693697929382324
42 3.8927364349365

In [41]:
# test the steering vector
base_prompt = "Generate a 100-150 word summary of the given transcript."
for tr in range(0,5): 
    print('BASELINE SUMMARY\n')
    print(outputs_personality['tf'][n_train+tr])
    prompt = base_prompt + '\n Transcript: ' + transcripts[n_train+tr]
    toks_test = tokenize_instructions(tokenizer, [{"role": "user", "content": prompt}])
    # steer to A
    generation_intervene, _ = steering_vectors.do_steering(model, toks_test.to(device), sv_optimised.to(device), scale = -1, layer = [20], proj=False, all_toks=False)
    intervene = tokenizer.decode(generation_intervene[0,toks_test.shape[1]:], skip_special_tokens=True)
    print('GENERATED SUMMARY\n')
    print(intervene)
    print('\n')

BASELINE SUMMARY

Alright, folks, grab your hard hats because we’re diving into the concrete jungle of tomorrow, and it’s less about skyscrapers and more about squatter shacks! Our speaker takes us on a whirlwind tour through the world’s most vibrant “neighborhoods” — think Kibera, Rio’s favelas, and Istanbul’s Sultanbelyi — where the real estate market is as stable as a Jenga tower in an earthquake. With a staggering 1.5 million people migrating to urban areas every week, it’s clear that the future is crowded. But don’t let the corrugated metal roofs fool you; these communities are brimming with hope, resilience, and, apparently, an unshakeable bond with their trash heaps. Our hero, Julius, shows us that while they may lack flush toilets, they have something more valuable: a sense of community and a whole lot of spirit. So, let’s ditch the “urban poverty” label and recognize these bustling hubs as the legit cities of the future—because who needs a mortgage when you can just squat?
GEN

##### Winston Churchill

In [43]:
# PERSONALITY WC
# dd_with_c_A = [[{"role": "user", "content": "Generate a 100-150 word summary of the given transcript. \n Transcript: " + x[:200]+'...'+x[-200:]}, {"role": "assistant", "content":y}] for (x,y) in zip(transcripts[:n_train],outputs_personality['wc'][:n_train])]
dd_with_c_A = [[{"role": "assistant", "content":y}] for y in outputs_personality['wc'][:n_train]]
toks_A = tokenize_instructions(tokenizer, dd_with_c_A).to(device)
sv_optimised = steering_vectors.optimise_steering_vecs(model, toks_A, None, hidden_dim=3072, layer=7, batch_size=6, num_epochs=10)

0 13.984513282775879
6 13.13253402709961
12 12.372668266296387
18 11.882608413696289
24 11.111212730407715
30 9.767813682556152
36 10.522783279418945
42 10.87966251373291
0 9.88300609588623
6 9.798846244812012
12 9.48879337310791
18 9.360311508178711
24 8.87406063079834
30 7.904541969299316
36 8.553435325622559
42 8.9813232421875
0 8.177122116088867
6 8.213834762573242
12 8.01321792602539
18 7.9455156326293945
24 7.676511287689209
30 6.955747127532959
36 7.539624214172363
42 7.974547386169434
0 7.23214054107666
6 7.349337577819824
12 7.158199787139893
18 7.138918876647949
24 6.940279960632324
30 6.327184200286865
36 6.837248802185059
42 7.233508110046387
0 6.560625076293945
6 6.736849784851074
12 6.571007251739502
18 6.527888774871826
24 6.377410888671875
30 5.831509590148926
36 6.277202129364014
42 6.652895927429199
0 6.013343334197998
6 6.209742546081543
12 6.064411640167236
18 6.031572341918945
24 5.91693639755249
30 5.421172142028809
36 5.8058929443359375
42 6.165076732635498
0 5.5

In [47]:
# test the steering vector
base_prompt = "Generate a 100-150 word summary of the given transcript."
for tr in range(0,1): 
    print('BASELINE SUMMARY\n')
    print(outputs_personality['wc'][tr])
    prompt = base_prompt + '\n Transcript: ' + transcripts[tr]
    toks_test = tokenize_instructions(tokenizer, [{"role": "user", "content": prompt}])
    # steer to A
    generation_intervene, _ = steering_vectors.do_steering(model, toks_test.to(device), sv_optimised.to(device), scale = -.3, layer = [7], proj=False, all_toks=False)
    intervene = tokenizer.decode(generation_intervene[0,toks_test.shape[1]:], skip_special_tokens=True)
    print('GENERATED SUMMARY\n')
    print(intervene)
    print('\n')

BASELINE SUMMARY

Ladies and gentlemen, esteemed colleagues, we stand at the precipice of a momentous epoch, wherein the clarion call of our climate crisis reverberates through the annals of human endeavor. In a gathering infused with the spirit of innovation and resolve, the venerable Al Gore has illuminated the path forward, urging us to embrace our collective responsibility toward a sustainable future. With a blend of candor and humor, he recounted the poignant transition from power to humility, epitomizing the profound shift that accompanies our shared journey. Yet, amidst laughter, he imparted grave truths about our planet's plight, beckoning us to act with urgency. He implores us to become stewards of change, champions of carbon neutrality, and catalysts of a movement that transcends political divides. Let us heed this call, for the fate of our world lies not in the hands of a few, but in the unwavering commitment of us all. Together, we shall forge a legacy of resilience and hop